In [31]:
#Imports
from __future__ import unicode_literals, print_function, division
from io import open
import wget # to download passwordlist
import glob
import os
import numpy as np
import unicodedata
import string
import torch
import torch.nn as nn
from torch.autograd import Variable

Auslesen der Passworddatei:

In [32]:
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in allowed_letters
    )

def readPasswords(filename):
    passwords = open(filename, encoding="utf8").read().split("\n")
    return [unicodeToAscii(password) for password in passwords]

def charToIndex(char):
    return allowed_letters.find(char)

def passwordToTensor(name):
    ret = torch.zeros(len(name), 1, password_len)
    for i, char in enumerate(name):
        ret[i][0][charToIndex(char)] = 1
    return ret

def targetToTensor(password):
    indizes = [letters.find(password[i]) for i in range(1,len(password))]
    indizes.append(password_len - 1)
    return torch.LongTensor(indizes)
                                                        
    
allowed_letters = string.ascii_letters + string.digits + string.punctuation
password_len = len(allowed_letters) + 1

Download der Passwortliste

In [33]:
#rockyou Liste produziert: 'utf-8' codec can't decode byte 0xf1 in position 5079963: invalid continuation byte
#filename = 'rockyou.txt'#Vermutung: Liste ist zu lang
filename = 'john.txt'

if os.path.isfile(filename):
    print('passwordlist exists')
    passwords = readPasswords(filename) #Pfad noch anpassen
else:
    print('download passwordlist:')
    #download list
    wget.download('https://www.scrapmaker.com/data/wordlists/dictionaries/rockyou.txt')
    passwords = readPasswords(filename) #Pfad noch anpassen

passwordlist exists


Generatorklasse:

In [34]:
#https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html
class PasswordGenerator(nn.Module):
    def __init__(self, input, hidden, output):
        super(PasswordGenerator, self).__init__()
        self.hidden = hidden
        self.input2hidden = nn.Linear(input + hidden, hidden)
        self.input2output = nn.Linear(input + output, output)
        self.output2output = nn.Linear(hidden +  output, output)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)
        
        
    def forward(self, input, hidden):
        combined_input = torch.cat((input, hidden), dim=1)
        hidden = self.input2hidden(combined_input)
        output = self.input2output(combined_input)
        combined_output = torch.cat((hidden, output), dim=1)
        output = self. output2output(combined_output)
        output = self.dropout(output)
        output = self.softmax(output)
        return hidden, output

    def initHidden(self):
        return Variable(torch.zeros(1, self.hidden))

Training:

In [35]:
model = PasswordGenerator(len(allowed_letters)+1, len(allowed_letters)+1, len(allowed_letters)+1)

criterion = nn.NLLLoss()
learning_rate = 0.001

def trainPasswords(input, target):
    hidden = model.initHidden()
    model.zero_grad()
    loss = 0
    for i in range(input.size()[0]): 
        print(input[i].shape)
        print(hidden.shape)
        output, hidden = model(input[i], hidden)
        loss += criterion(output, target[i])
        
    loss.backward()
    for p in model.parameters():
        p.data = p.data.add(-learning_rate, p.grad.data)
        
    return output, loss



loss_sum = 0
plots = []

for i in range(1, 2):
    for j in range(len(passwords)):
        password = passwords[j]
        print(password)
        input = Variable(passwordToTensor(password))
        target = Variable(passwordToTensor(password))
    
        output, loss = trainPasswords(input, target)
        sum_loss += loss.data[0]/input.size()[0]
        


print("SAMPLE: ", sample())                                      

12345
torch.Size([1, 95])
torch.Size([1, 95])


RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'target'

In [ ]:
def prepareData():
    lines = []
    f = open(filename,encoding="utf-8").read().split('\n')
    for line in f:
        lines.append(line)
        
    return lines


    
#prepareData()


Mit LSTM:

Deklarierung der Variablen:

In [ ]:
lstm_inputs = 10 # kann geaendert werden. 1 = 1 input pro timestep
hidden_layers = 32
batch_size = 10#Batch size noch berechnen
model_output = 1
model_layers = 2
iterations = 10000

LSTM-Generator:

In [ ]:
class PasswordLSTMGenerator(nn.Module):
    def __init__(self, input_lstm, hiddens, output ):
        super(PasswordGenerator, self).__init__()
        self.input_len = input_lstm
        self.hiddens = hiddens
        self.output = output
        
        self.lstm()
    
    
    def __init__hiddens(self):
        hiddens = torch.zeros(self.layers, self.batch, self.hidden
        return (hiddens, hiddens)
                                         
                          
    def forward_pass(self, input):
        output_lstm, self.hiddens = self.lstm(input.view(len(input), self.batch, -1))
        pred = self.linear(output_lstm[-1].view(self.batch,-1))
    
        return y_pred.view(-1)
    
    model = LSTM(lstm_inputs, hidden_layers, batch= batch_size, output = model_output, layers = model_layers)

LSMT-Training:

In [ ]:
#Train
history = np.zeros(iterations)

for i in range(iterations):
    model.hidden = model.init_hidden()
    
    pred = model